In [11]:
import pandas as pd
import networkx as nx
import json
from dowhy import CausalModel
import pickle
import logging
logging.basicConfig(level=logging.WARNING)

In [32]:
with open('../graphs/full_causal.gpickle', 'rb') as f:
    causal_graph = pickle.load(f)

data = pd.read_csv('/home/paperspace/EQLS-Causal-Project/data/eqls_processed.csv')
data = data[list(causal_graph.nodes)]

In [33]:
model = CausalModel(
    data=data,
    treatment="Y11_Q57",
    outcome="Y11_MWIndex",
    graph=nx.nx_pydot.to_pydot(causal_graph).to_string()
)

In [34]:
estimand = model.identify_effect()
print("Identified estimand:", estimand)

Identified estimand: Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                         ↪
───────────(E[Y_11_MWIndex|Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2 ↪
d[Y_11_Q57]                                                                    ↪

↪                                                           
↪ a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32])
↪                                                           
Estimand assumption 1, Unconfoundedness: If U→{Y11_Q57} and U→Y11_MWIndex then P(Y11_MWIndex|Y11_Q57,Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32,U) = P(Y11_MWIndex|Y11_Q57,Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: fr

In [37]:
estimate = model.estimate_effect(estimand,
                                 method_name="backdoor.propensity_score_matching")
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                         ↪
───────────(E[Y_11_MWIndex|Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2 ↪
d[Y_11_Q57]                                                                    ↪

↪                                                           
↪ a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32])
↪                                                           
Estimand assumption 1, Unconfoundedness: If U→{Y11_Q57} and U→Y11_MWIndex then P(Y11_MWIndex|Y11_Q57,Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32,U) = P(Y11_MWIndex|Y11_Q57,Y11_EmploymentStatus,Y11_HHstructure,Y11_Q7,Y11_HH2a,Y11_HHsize,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_Q32)

## Realized estimand
b: Y11_MWIndex~Y11_Q57+Y11_EmploymentStatus+Y11_H

In [40]:
refute_results = model.refute_estimate(estimand, estimate,
                                       method_name="placebo_treatment_refuter")
print(refute_results)

Refute: Use a Placebo Treatment
Estimated effect:4.270977323356372
New effect:-0.00532130005140801
p value:0.9



In [41]:
refute_results = model.refute_estimate(estimand, estimate,
                                       method_name="data_subset_refuter",)
print(refute_results)

Refute: Use a subset of data
Estimated effect:4.270977323356372
New effect:4.412831643581323
p value:0.5800000000000001

